In [14]:
%pip install llama-index-readers-wikipedia
%pip install llama-index
%pip install wikipedia
%pip install faiss-cpu
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor
# %pip install llama-index-llms-llama-cpp

%pip install llama-index-vector-stores-faiss


In [15]:
from llama_index.readers.wikipedia import WikipediaReader


In [16]:
reader = WikipediaReader()

In [17]:
cities = ["จังหวัดอยุธยา",
        #   "จังหวัดปทุมธานี",
        #   "จังหวัดนครราชสีมา",
        #   "จังหวัดเชียงใหม่",
          "จังหวัดนครศรีธรรมราช",]

In [18]:
import wikipedia
wikipedia.set_lang('th')

In [20]:
documents = reader.load_data(pages=cities)

In [21]:
documents

[Document(id_='4203', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='พระนครศรีอยุธยา เป็นจังหวัดหนึ่งในภาคกลางและเป็นเขตเศรษฐกิจอุตสาหกรรมที่สำคัญ โดยมีผลิตภัณฑ์มวลรวมของจังหวัดมีมูลค่าสูงเป็นอันดับ 3 ของประเทศ และมีประวัติศาสตร์ความเป็นมาที่ยาวนาน เคยมีชื่อเสียงเป็นแหล่งปลูกข้าวที่สำคัญ จังหวัดพระนครศรีอยุธยาเป็นจังหวัดเดียวในประเทศไทยที่ไม่มีอำเภอเมือง แต่มี อำเภอพระนครศรีอยุธยา เป็นศูนย์กลางการบริหารจัดการด้านต่าง ๆ ชาวบ้านโดยทั่วไปนิยมเรียกอีกชื่อว่า "กรุงเก่า" หรือ "เมืองกรุงเก่า" ห่างจากกรุงเทพมหานครประมาณ 75 กิโลเมตร\n\n\n== ประวัติศาสตร์ ==\n\nพระนครศรีอยุธยาเคยเป็นราชธานี (เมืองหลวง) ของอาณาจักรอยุธยา หรืออาณาจักรสยาม ตลอดระยะเวลา 417 ปี ตั้งแต่ พ.ศ. 1893 กระทั่งเสียกรุงแก่พม่า เมื่อ พ.ศ. 2310 ครั้นเมื่อพระเจ้าตากสินมหาราชทรงสถาปนาราชธานีแห่งใหม่ที่กรุงธนบุรี กรุงศรีอยุธยาก็ไม่ได้กลายเป็นเมืองร้าง ยังมีคนที่รักถิ่นฐานบ้านเดิมอาศัยอยู่และมีราษฎรที่หลบหนีไปอยู่ตามป่ากลับเข้ามาอาศัยอยู่รอบ ๆ เมือง รวมกันเข้าเป็

In [22]:
import faiss

# dimensions of BAAI/bge-m3
d = 1024
faiss_index = faiss.IndexFlatIP(d)

In [23]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [24]:
embedding_model_name = '/Users/9meo/Dev/llm/bge-m3'
# embedding_model_name = 'kornwtp/simcse-model-phayathaibert'
embed_model = HuggingFaceEmbedding(model_name=embedding_model_name,max_length=1024, device='cpu')

In [25]:
from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
)        
from llama_index.vector_stores.faiss import FaissVectorStore

In [26]:
from llama_index.core.node_parser import TokenTextSplitter
splitter = TokenTextSplitter(
    chunk_size=256,
    chunk_overlap=10,
    separator=" ",
)

In [27]:
nodes = splitter.get_nodes_from_documents(
    documents, show_progress=True
)

Parsing nodes: 100%|██████████| 2/2 [00:00<00:00, 27.69it/s]


In [28]:
nodes

[TextNode(id_='e4bb77fc-601b-44b0-b5de-184ba7f0b513', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='4203', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='90b578b743de68c680e945bda9e8935b1df973e46aeea118e519fad36a64c080'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='904cec73-2c2e-47e5-845b-621005727508', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='512a1e78b324df81a579f05d814477a8f486a5654e5509e1efeac3686a1f1427')}, text='พระนครศรีอยุธยา เป็นจังหวัดหนึ่งในภาคกลางและเป็นเขตเศรษฐกิจอุตสาหกรรมที่สำคัญ โดยมีผลิตภัณฑ์มวลรวมของจังหวัดมีมูลค่าสูงเป็นอันดับ 3 ของประเทศ และมีประวัติศาสตร์ความเป็นมาที่ยาวนาน เคยมีชื่อเสียงเป็นแหล่งปลูกข้าวที่สำคัญ', start_char_idx=0, end_char_idx=219, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 TextNode(id_='904cec73-2c2e-47e5-845b-621005727508', embeddin

In [29]:
from llama_index.core import Settings

Settings.llm = None
Settings.embed_model = embed_model

LLM is explicitly disabled. Using MockLLM.


In [30]:
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(nodes=nodes,storage_context=storage_context)

In [31]:
index.storage_context.persist(persist_dir='../index')
